/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT *
FROM Facilities
WHERE membercost >0;

/* Q2: How many facilities do not charge a fee to members? */

4 facilities do not charge members for use.

SELECT COUNT( * )
FROM Facilities
WHERE membercost =0;

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost >0
AND membercost < ( .2 * monthlymaintenance );


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT *
FROM Facilities
WHERE facid
IN ( 1, 5 );


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance >100
THEN 'expensive'
ELSE 'cheap'
END AS cost
FROM Facilities;

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

Order members by joindate in descending order.

SELECT firstname, surname
FROM Members
ORDER BY joindate DESC;

Select the last member to join.

SELECT firstname, surname
FROM Members
WHERE joindate = (
SELECT MAX( joindate )
FROM Members );

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT
    DISTINCT(CONCAT(m.firstname, ' ', m.surname)) AS memname,
    CASE WHEN b.facid IN (0,1) THEN 'Tennis Court'
    ELSE NULL END AS tenniscourt
FROM Bookings AS b
LEFT JOIN Members AS m
ON b.memid = m.memid
LEFT JOIN Facilities AS f
ON b.facid = f.facid
WHERE b.facid IN (0,1);

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT 
    f.name,
    CONCAT(m.firstname, ' ', m.surname) AS memname,
    CASE WHEN b.memid = 0 AND f.guestcost * b.slots > 30 THEN f.guestcost * b.slots
         WHEN b.memid != 0 AND f.membercost * b.slots > 30 THEN f.membercost * b.slots 
         END AS cost
FROM Bookings as b
LEFT JOIN Members AS m
ON b.memid = m.memid
LEFT JOIN Facilities AS f
ON b.facid = f.facid
WHERE CASE WHEN b.memid = 0 AND f.guestcost * b.slots > 30 THEN f.guestcost * b.slots
         WHEN b.memid != 0 AND f.membercost * b.slots > 30 THEN f.membercost * b.slots 
         END IS NOT NULL
    AND starttime BETWEEN '2012-09-14' AND '2012-09-15'
ORDER BY cost DESC;

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT
	f.name, 
	CONCAT( m.firstname, ' ', m.surname ) AS memname, 
	COALESCE((b.slots * (
		SELECT guestcost
		FROM Facilities AS f
		WHERE f.facid = b.facid
		AND b.memid =0 )), 
	(b.slots *
		(SELECT  membercost
		FROM Facilities AS f
		WHERE f.facid = b.facid
		AND b.memid !=0 ))) AS cost
FROM Bookings AS b
INNER JOIN Facilities AS f ON b.facid = f.facid
INNER JOIN Members AS m ON b.memid = m.memid
WHERE starttime BETWEEN '2012-09-14' AND '2012-09-15' AND COALESCE((b.slots * (
		SELECT guestcost
		FROM Facilities AS f
		WHERE f.facid = b.facid
		AND b.memid =0 )), 
	(b.slots *
		(SELECT  membercost
		FROM Facilities AS f
		WHERE f.facid = b.facid
		AND b.memid !=0 ))) > 30
ORDER BY cost DESC

/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


/* Q12: Find the facilities with their usage by member, but not guests */


/* Q13: Find the facilities usage by month, but not guests */



In [3]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3
from sqlite3 import Error

In [47]:
#Q10
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT * 
        FROM
            (SELECT 
             f.name,
             SUM(CASE WHEN b.memid = 0 THEN b.slots * f.guestcost
             WHEN b.memid != 0 THEN b.slots * f.membercost 
             END) AS revenue
        FROM FACILITIES AS f
        LEFT JOIN BOOKINGS AS b
        ON f.facid = b.facid
        GROUP BY f.facid) AS sub
        WHERE sub.revenue < 1000
        ORDER BY sub.revenue
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


In [48]:
#Q11
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT surname||', '||firstname AS member,
                recommendedby
        FROM Members
        ORDER BY member        
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Bader, Florence', '9')
('Baker, Anne', '9')
('Baker, Timothy', '13')
('Boothe, Tim', '3')
('Butters, Gerald', '1')
('Coplin, Joan', '16')
('Crumpet, Erica', '2')
('Dare, Nancy', '4')
('Farrell, David', '')
('Farrell, Jemima', '')
('GUEST, GUEST', '')
('Genting, Matthew', '5')
('Hunt, John', '30')
('Jones, David', '4')
('Jones, Douglas', '11')
('Joplette, Janice', '1')
('Mackenzie, Anna', '1')
('Owen, Charles', '1')
('Pinker, David', '13')
('Purview, Millicent', '2')
('Rownam, Tim', '')
('Rumney, Henrietta', '20')
('Sarwin, Ramnaresh', '15')
('Smith, Darren', '')
('Smith, Darren', '')
('Smith, Jack', '1')
('Smith, Tracy', '')
('Stibbons, Ponder', '6')
('Tracy, Burton', '')
('Tupperware, Hyacinth', '')
('Worthington-Smyth, Henry', '2')


In [50]:
#Q12
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT
          f.name,
          COUNT(b.bookid) AS memusage
        FROM FACILITIES AS f
        LEFT JOIN BOOKINGS as b
        ON f.facid = b.facid
        WHERE b.memid != 0
        GROUP BY f.facid       
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Tennis Court 1', 308)
('Tennis Court 2', 276)
('Badminton Court', 344)
('Table Tennis', 385)
('Massage Room 1', 421)
('Massage Room 2', 27)
('Squash Court', 195)
('Snooker Table', 421)
('Pool Table', 783)


In [53]:
#Q13
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT
          f.name,
          COUNT(b.bookid) AS monthusage,
          strftime("%m", starttime) AS month
        FROM FACILITIES AS f
        LEFT JOIN BOOKINGS as b
        ON f.facid = b.facid
        WHERE b.memid != 0
        GROUP BY month       
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Table Tennis', 480, '07')
('Tennis Court 1', 1168, '08')
('Tennis Court 1', 1512, '09')
